In [1]:
import requests
import math


In [ ]:
def round_up_nearest_cent(number):
  """
  Rounds a number up to the nearest hundredth.
  """
  return math.ceil(number * 100) / 100

def get_orderbook_url(market_ticker):
    return f"https://api.elections.kalshi.com/trade-api/v2/markets/{market_ticker}/orderbook"

def get_prices(orderbook_url):
  orderbook_response = requests.get(orderbook_url)
  orderbook_data = orderbook_response.json()

  highest_bid = orderbook_data['orderbook']['yes'][-1][0]
  highest_bid_volume = orderbook_data['orderbook']['yes'][-1][1]

  lowest_ask = 100 - orderbook_data['orderbook']['no'][-1][0]
  lowest_ask_volume = orderbook_data['orderbook']['no'][-1][1]

  return highest_bid, highest_bid_volume, lowest_ask, lowest_ask_volume

def get_prices_from_market(market):
    market_ticker = market['ticker']
    orderbook_url = get_orderbook_url(market_ticker)
    yes_bid, yes_bid_volume, yes_ask, yes_ask_volume = get_prices(orderbook_url)
    print(f"- {market['ticker'].split('-')[-1]}: Yes Bid: {yes_bid}¢ | Volume: {yes_bid_volume}, Yes Ask: {yes_ask}¢ | Volume: {yes_ask_volume}")
    return yes_bid, yes_bid_volume, yes_ask, yes_ask_volume

# Calculates the fair price to buy/sell a given number of contracts at the lowest ask/highest bid, including fees.
# Returns (taker_price, maker_price, taker_probability, maker_probability)
def fair_price(number_of_contracts, lowest_ask):
    lowest_ask_percentage = lowest_ask / 100
    highest_bid = lowest_ask - 1
    highest_bid_percentage = highest_bid / 100
    taker_odds = 0.07 * lowest_ask_percentage * (1 - lowest_ask_percentage)
    maker_odds = 0.0175 * highest_bid_percentage * (1 - highest_bid_percentage)
    taker_fees = round_up_nearest_cent(taker_odds * number_of_contracts)
    maker_fees = round_up_nearest_cent(maker_odds * number_of_contracts)

    return round(taker_fees + number_of_contracts * lowest_ask_percentage, 2),  round(maker_fees + number_of_contracts * highest_bid_percentage, 2), round(taker_odds + lowest_ask_percentage, 6), round(maker_odds + highest_bid_percentage, 6)

In [12]:
url = "https://api.elections.kalshi.com/trade-api/v2/series/KXNFLGAME"
response = requests.get(url)
series_data = response.json()

print(f"Series Title: {series_data['series']['title']}")
print(f"Frequency: {series_data['series']['frequency']}")
print(f"Category: {series_data['series']['category']}")



Series Title: Professional Football Game
Frequency: custom
Category: Sports


In [20]:
# Get all markets for the KXNFLGAME series
markets_url = f"https://api.elections.kalshi.com/trade-api/v2/markets?series_ticker=KXNFLGAME&status=open"
markets_response = requests.get(markets_url)
markets_data = markets_response.json()


# markets_data
print(f"\nActive markets in KXNFLGAME series:")
for market in markets_data['markets']:
    print(f"- {market['ticker']}: {market['title']}")
    print(f"  Event: {market['event_ticker']}")
    print(f"  Yes Bid: {market['yes_bid']}¢, Yes Ask: {market['yes_bid']}¢ | Volume: {market['volume']}")
    print()

# Get details for a specific event if you have its ticker
if markets_data['markets']:
    # Let's get details for the first market's event
    event_ticker = markets_data['markets'][0]['event_ticker']
    event_url = f"https://api.elections.kalshi.com/trade-api/v2/events/{event_ticker}"
    event_response = requests.get(event_url)
    event_data = event_response.json()

    print(f"Event Details:")
    print(f"Title: {event_data['event']['title']}")
    print(f"Category: {event_data['event']['category']}")


Active markets in KXNFLGAME series:
- KXNFLGAME-25SEP29CINDEN-DEN: Cincinnati at Denver Winner?
  Event: KXNFLGAME-25SEP29CINDEN
  Yes Bid: 71¢, Yes Ask: 71¢ | Volume: 0

- KXNFLGAME-25SEP29CINDEN-CIN: Cincinnati at Denver Winner?
  Event: KXNFLGAME-25SEP29CINDEN
  Yes Bid: 28¢, Yes Ask: 28¢ | Volume: 0

- KXNFLGAME-25SEP29NYJMIA-NYJ: New York J at Miami Winner?
  Event: KXNFLGAME-25SEP29NYJMIA
  Yes Bid: 41¢, Yes Ask: 41¢ | Volume: 0

- KXNFLGAME-25SEP29NYJMIA-MIA: New York J at Miami Winner?
  Event: KXNFLGAME-25SEP29NYJMIA
  Yes Bid: 58¢, Yes Ask: 58¢ | Volume: 0

- KXNFLGAME-25SEP28GBDAL-GB: Green Bay at Dallas Winner?
  Event: KXNFLGAME-25SEP28GBDAL
  Yes Bid: 68¢, Yes Ask: 68¢ | Volume: 64

- KXNFLGAME-25SEP28GBDAL-DAL: Green Bay at Dallas Winner?
  Event: KXNFLGAME-25SEP28GBDAL
  Yes Bid: 30¢, Yes Ask: 30¢ | Volume: 0

- KXNFLGAME-25SEP28BALKC-KC: Baltimore at Kansas City Winner?
  Event: KXNFLGAME-25SEP28BALKC
  Yes Bid: 42¢, Yes Ask: 42¢ | Volume: 0

- KXNFLGAME-25SEP28BALKC-

In [31]:
group_by_title = {}

for market in markets_data['markets']:
    title = market['title']
    if title not in group_by_title:
        group_by_title[title] = []
    group_by_title[title].append(market)

for title, markets in reversed(group_by_title.items()):
    print(f"\nMarkets for {title}:")

    market_a = markets[0]
    a_name = market_a['ticker'].split('-')[-1]
    a_yes_bid, a_yes_bid_volume, a_yes_ask, a_yes_ask_volume = get_prices_from_market(market_a)

    market_b = markets[1]
    b_name = market_b['ticker'].split('-')[-1]
    b_yes_bid, b_yes_bid_volume, b_yes_ask, b_yes_ask_volume = get_prices_from_market(market_b)

    a_lowest_yes_bid = min(a_yes_bid, 100 - b_yes_ask)
    a_lowest_yes_ask = min(a_yes_ask, 100 - b_yes_bid)
    b_lowest_yes_bid = min(b_yes_bid, 100 - a_yes_ask)
    b_lowest_yes_ask = min(b_yes_ask, 100 - a_yes_bid)

    
    a_taker_price, a_maker_price, a_taker_probability, a_maker_probability = fair_price(100, a_lowest_yes_ask)
    b_taker_price, b_maker_price, b_taker_probability, b_maker_probability = fair_price(100, b_lowest_yes_ask)

    # print(f"{a_name}: Lowest price YES Bid: {a_lowest_yes_bid}, YES Ask: {a_lowest_yes_ask}")
    # print(f"{b_name}: Lowest price YES Bid: {b_lowest_yes_bid}, YES Ask: {b_lowest_yes_ask}")

    print(f"{a_name}: Taker Price: ${a_taker_price} ({a_taker_probability}), Maker Price: ${a_maker_price} ({a_maker_probability})")
    print(f"{b_name}: Taker Price: ${b_taker_price} ({b_taker_probability}), Maker Price: ${b_maker_price} ({b_maker_probability})")

    
    


Markets for Miami at Buffalo Winner?:
- MIA: Yes Bid: 18¢ | Volume: 539847, Yes Ask: 20¢ | Volume: 521193
- BUF: Yes Bid: 80¢ | Volume: 256621, Yes Ask: 82¢ | Volume: 605759
MIA: Taker Price: $21.13 (0.2112), Maker Price: $19.27 (0.192693)
BUF: Taker Price: $83.04 (0.830332), Maker Price: $81.27 (0.812693)

Markets for Indianapolis at Tennessee Winner?:
- IND: Yes Bid: 65¢ | Volume: 304322, Yes Ask: 66¢ | Volume: 252583
- TEN: Yes Bid: 34¢ | Volume: 60693, Yes Ask: 35¢ | Volume: 340739
IND: Taker Price: $67.58 (0.675708), Maker Price: $65.4 (0.653981)
TEN: Taker Price: $36.6 (0.365925), Maker Price: $34.4 (0.343927)

Markets for Pittsburgh at New England Winner?:
- PIT: Yes Bid: 55¢ | Volume: 145633, Yes Ask: 56¢ | Volume: 614159
- NE: Yes Bid: 45¢ | Volume: 398, Yes Ask: 46¢ | Volume: 203090
PIT: Taker Price: $56.74 (0.567325), Maker Price: $54.44 (0.544347)
NE: Taker Price: $46.74 (0.467325), Maker Price: $44.44 (0.444312)

Markets for New York J at Tampa Bay Winner?:
- TB: Yes Bid:

In [8]:
# Get orderbook for a specific market
# Replace with an actual market ticker from the markets list
market_ticker = markets_data['markets'][-1]['ticker']
orderbook_url = f"https://api.elections.kalshi.com/trade-api/v2/markets/{market_ticker}/orderbook"

orderbook_response = requests.get(orderbook_url)
orderbook_data = orderbook_response.json()



print(f"\nOrderbook for {market_ticker}:")

print("\nYES ASKS:")
for bid in orderbook_data['orderbook']['no'][-3:]:  # Show top 5
    print(f"  Price: {100 - bid[0]}¢, Quantity: {bid[1]}")

print("\nYES BIDS:")
for bid in reversed(orderbook_data['orderbook']['yes'][-3:]):  # Show top 5
    print(f"  Price: {bid[0]}¢, Quantity: {bid[1]}")

    
market_ticker = markets_data['markets'][-2]['ticker']
orderbook_url = f"https://api.elections.kalshi.com/trade-api/v2/markets/{market_ticker}/orderbook"

orderbook_response = requests.get(orderbook_url)
orderbook_data = orderbook_response.json()



print(f"\nOrderbook for {market_ticker}:")

print("\nYES ASKS:")
for bid in orderbook_data['orderbook']['no'][-1:]:  # Show top 5
    print(f"  Price: {100 - bid[0]}¢, Quantity: {bid[1]}")

print("\nYES BIDS:")
for bid in reversed(orderbook_data['orderbook']['yes'][-1:]):  # Show top 5
    print(f"  Price: {bid[0]}¢, Quantity: {bid[1]}")


Orderbook for KXNFLGAME-25SEP18MIABUF-BUF:

YES ASKS:
  Price: 90¢, Quantity: 2365471
  Price: 89¢, Quantity: 4451200
  Price: 88¢, Quantity: 5004840

YES BIDS:
  Price: 87¢, Quantity: 363
  Price: 86¢, Quantity: 724921
  Price: 85¢, Quantity: 655602

Orderbook for KXNFLGAME-25SEP18MIABUF-MIA:

YES ASKS:
  Price: 14¢, Quantity: 3339564

YES BIDS:
  Price: 12¢, Quantity: 1589158


In [68]:
print(fair_price(100, 14))
print(fair_price(100, 88))

(14.85, 13.2, 0.148428, 0.131979)
(88.74, 87.2, 0.887392, 0.871979)
